In [ ]:


from __future__ import annotations

import os
import time
import numpy as np
import tensorflow as tf

from mec_env_var import MecTermLD, MecSvrEnv


def _setup_tf():
    
    gpus = tf.config.list_physical_devices('GPU')
    for gpu in gpus:
        try:
            tf.config.experimental.set_memory_growth(gpu, True)
        except Exception:
            pass


def main():
    _setup_tf()

    for k in range(10):
        print('---------' + str(k) + '------------')

        MAX_EPISODE = 100
        MAX_EPISODE_LEN = 10000

        NUM_R = 4
        SIGMA2 = 1e-9

        t_factor = 0.5

        config = {'state_dim': 3, 'action_dim': 2}
        train_config = {
            'sigma2': SIGMA2,
            'random_seed': int(time.time() * 1000) % 1000,
        }

        res_path = 'test_results_ddpg_pure_training/'
        model_root = 'models_ddpg_pure_training/'
        model_prefix = os.path.join(model_root, f"train_model_{k}-1500")

        user_config = [
            {'id': '1', 'model': 'AR', 'num_r': NUM_R, 'rate': 1.0, 'dis': 100, 'action_bound': 2,
             'data_buf_size': 100, 't_factor': t_factor, 'penalty': 1000},
            {'id': '2', 'model': 'AR', 'num_r': NUM_R, 'rate': 2.0, 'dis': 100, 'action_bound': 2,
             'data_buf_size': 100, 't_factor': t_factor, 'penalty': 1000},
            {'id': '3', 'model': 'AR', 'num_r': NUM_R, 'rate': 3.0, 'dis': 100, 'action_bound': 2,
             'data_buf_size': 100, 't_factor': t_factor, 'penalty': 1000},
        ]

        print(user_config)

        
        user_list = []
        for info in user_config:
            info = dict(info)  
            info.update(config)
            info['ckpt_dir'] = f"{model_prefix}_{info['id']}"
            
            
            if not os.path.exists(info['ckpt_dir']):
                print(f"WARNING: Checkpoint directory not found: {info['ckpt_dir']}")
                print(f"Skipping model {k}")
                break
            
            user_list.append(MecTermLD(info, train_config))
            print('Initialization OK!----> user ' + info['id'])
        
        
        if len(user_list) != len(user_config):
            continue

        
        env = MecSvrEnv(user_list, NUM_R, SIGMA2, MAX_EPISODE_LEN)

        res_r = []
        res_p = []
        res_b = []
        res_o = []
        res_d = []

        
        for i in range(MAX_EPISODE):
            cur_init_ds_ep = env.reset(isTrain=False)

            cur_r_ep = np.zeros(len(user_list), dtype=np.float32)
            cur_p_ep = np.zeros(len(user_list), dtype=np.float32)
            cur_op_ep = np.zeros(len(user_list), dtype=np.float32)
            cur_ts_ep = np.zeros(len(user_list), dtype=np.float32)
            cur_ps_ep = np.zeros(len(user_list), dtype=np.float32)
            cur_rs_ep = np.zeros(len(user_list), dtype=np.float32)
            cur_ds_ep = np.zeros(len(user_list), dtype=np.float32)
            cur_ch_ep = np.zeros(len(user_list), dtype=np.float32)
            cur_of_ep = np.zeros(len(user_list), dtype=np.float32)

            for j in range(MAX_EPISODE_LEN):
                
                (cur_r, done, cur_p, cur_op, _noise, cur_ts, cur_ps, cur_rs,
                 cur_ds, cur_ch, cur_of) = env.step_transmit(isRandom=False)

                cur_r_ep += cur_r
                cur_p_ep += cur_p
                cur_op_ep += cur_op
                cur_ts_ep += cur_ts
                cur_ps_ep += cur_ps
                cur_rs_ep += cur_rs
                cur_ds_ep += cur_ds
                cur_ch_ep += cur_ch
                cur_of_ep += cur_of

                if done:
                    res_r.append(cur_r_ep / MAX_EPISODE_LEN)
                    res_p.append(cur_p_ep / MAX_EPISODE_LEN)
                    res_b.append(cur_ds_ep / MAX_EPISODE_LEN)
                    res_o.append(cur_of_ep / MAX_EPISODE_LEN)
                    res_d.append(cur_ds)

                    print('%d:r:%s,p:%s,op:%s,tr:%s,pr:%s,rev:%s,dbuf:%s,ch:%s,ibuf:%s,rbuf:%s' %
                          (i,
                           cur_r_ep / MAX_EPISODE_LEN,
                           cur_p_ep / MAX_EPISODE_LEN,
                           cur_op_ep / MAX_EPISODE_LEN,
                           cur_ts_ep / MAX_EPISODE_LEN,
                           cur_ps_ep / MAX_EPISODE_LEN,
                           cur_rs_ep / MAX_EPISODE_LEN,
                           cur_ds_ep / MAX_EPISODE_LEN,
                           cur_ch_ep / MAX_EPISODE_LEN,
                           cur_init_ds_ep,
                           cur_ds))
                    break

        
        os.makedirs(res_path, exist_ok=True)
        timestamp = time.strftime("%b_%d_%Y_%H_%M_%S", time.localtime(time.time()))
        name = os.path.join(res_path, f'test_model_{k}_{timestamp}')
        np.savez(name, np.array(res_r), np.array(res_p), np.array(res_b), np.array(res_o), np.array(res_d))

        print("Saved test results:", name)
        print("Model tested:", model_prefix)




In [ ]:
main()
# 261m cho 5 model